<a href="https://colab.research.google.com/github/LukasEder1/StationarityPrediction/blob/main/LLMS/LLama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/LukasEder1/StationarityPrediction.git
!pip install datasets

fatal: destination path 'StationarityPrediction' already exists and is not an empty directory.


In [5]:
import pandas as pd

df = pd.read_csv("/content/StationarityPrediction/LLMS/clean_dataset_combined.csv", delimiter=";")

In [6]:
# Getting Rid of -1 Predicitions:
mask = df['final_vote'] == -1

# select all rows except the ones that contain 'Coca Cola'
df = df[~mask]

df = df.copy()
# binarize the classification variable
df.loc[:, "final_vote"]= df["final_vote"].apply(lambda x: int(x != 0))

In [7]:
from sklearn.model_selection import train_test_split

data = df[["text", "final_vote"]].rename(columns={"final_vote": "label"})

# Split Data into 80% train, 10% validation, 10% test
train_data, test_val_data = train_test_split(data, test_size=0.2, random_state=12, stratify=data['label'])

test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=12, stratify=test_val_data['label'])

print(f"Training set: {len(train_data)}")
print(f"Testing set: {len(test_data)}")
print(f"Validation set: {len(val_data)}")

Training set: 2304
Testing set: 288
Validation set: 289


In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from tqdm.auto import tqdm
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
txts = test_data.text.values

In [ ]:
input = f"""<s>[INST] <<SYS>>
Imagine you are deployed as a system that detecty wheter or not an sentence contains time-sensitive information or not.
Your name is Time Stationarity detector. Your main goal is to flag sentences with time sensitive infromation as such.
So you should only function as a binary classifier.
Take the following examples as a guideline: Stationary: 'i hate feeling like im forcing someone to talk to me',
Time-Sensitive: "I got 30 mins to decide if i want to participate in my super busy Monday or lay in bed and rest more'
<</SYS>>

{txts[0]} [/INST]"""

In [ ]:
pipeline(
        input,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
        return_full_text=False,
        temperature=0.5
    )

In [ ]:
def encode_labels(labels):
  return [int(label == "time-sensitive") for label in labels]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

labels = ["stationary",  "time-sensitive"]

y_preds = encode_labels(top_pred)
y_valid = test_data["label"].values

cm = confusion_matrix(y_preds, y_valid, labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=labels)
disp.plot()
plt.show()